In [1]:
import numpy as np
import matplotlib.pyplot as plt
from receptive_field_filter import *

num_neurons = 5
image_size = (16, 16)

base_rate = 2.0  # Base firing rate in Hz
gain = 1.0      # Gain factor for nonlinearity
bias = 0.0       # Bias term for nonlinearity
dt = 0.003        # Time bin size in seconds (3 ms)
random_seed_stim = 48691321

duration_rec = 6_000 # Duration of recording in seconds
pattern_duration = 0.009 # Duration of each pattern in seconds
# pattern duration should be prefearably a multiple of dt
n_show_pattern = np.ceil(pattern_duration / dt).astype(int)
n_patter_rec = np.ceil(duration_rec / pattern_duration).astype(int)

# Gauss

In [2]:
rng_stimulus = np.random.default_rng(random_seed_stim)
# stimulus = rng_stimulus.random((n_patter_rec, image_size[0], image_size[1]))
stimulus = rng_stimulus.integers(0, 2, (n_patter_rec, image_size[0], image_size[1]))
stimulus = 2 * stimulus - 1
# stimulus = rng_stimulus.uniform(0, 1, (n_patter_rec, image_size[0], image_size[1]))

stimulus = np.repeat(stimulus, n_show_pattern, axis=0)
assert stimulus.shape == (n_patter_rec*n_show_pattern, image_size[0], image_size[1])
print(stimulus.shape)

filter = gaussian(size=16, center=(8, 8), amplitude=1.0, sigma=1.0)
stimulus_filtered = stimulus * filter
assert np.array_equal(stimulus_filtered[0], stimulus[0] * filter)
stimulus_filtered.shape

stimulus_filtered_flat = stimulus_filtered.reshape(stimulus_filtered.shape[0], -1)
linear_response = stimulus_filtered_flat @ filter.flatten().T
assert linear_response.shape == (n_patter_rec*n_show_pattern,)
linear_response.shape

nl_firing_rate = base_rate * np.exp(gain * linear_response + bias)


spike_probs = nl_firing_rate * dt
spike_train = rng_stimulus.random(spike_probs.shape) < spike_probs
# spike_times_binned = spike_train.astype(int) * time_ticks

time_ticks = np.linspace(0.001, duration_rec, spike_train.shape[0], endpoint=False)
# not starting directly at 0, as this would be unrealistic in real experiments
spike_times = time_ticks[spike_train]

np.save('stimulus_gauss_1.npy', stimulus)
np.save('filter_gauss_1.npy', filter)
np.save('spike_train_gauss_1.npy', spike_train)

(2000001, 16, 16)


# Ricker

In [3]:
# image_size = (16, 16)

# base_rate = 1.0  # Base firing rate in Hz
# gain = 1.0      # Gain factor for nonlinearity
# bias = 0.0       # Bias term for nonlinearity
# dt = 0.003        # Time bin size in seconds (3 ms)
# random_seed_stim = 897465132

# duration_rec = 3_000 # Duration of recording in seconds
# pattern_duration = 0.009 # Duration of each pattern in seconds
# # pattern duration should be prefearably a multiple of dt
# n_show_pattern = np.ceil(pattern_duration / dt).astype(int)
# n_patter_rec = np.ceil(duration_rec / pattern_duration).astype(int)

In [4]:
rng_stimulus = np.random.default_rng(random_seed_stim)
stimulus = rng_stimulus.integers(0, 2, (n_patter_rec, image_size[0], image_size[1]))
stimulus = 2 * stimulus - 1
# stimulus = rng_stimulus.uniform(0, 1, (n_patter_rec, image_size[0], image_size[1]))
stimulus = np.repeat(stimulus, n_show_pattern, axis=0)
filter = ricker_wavelet(size=16, center=(14, 8), amplitude=3.0, sigma=0.99)
stimulus_filtered = stimulus * filter

stimulus_filtered_flat = stimulus_filtered.reshape(stimulus_filtered.shape[0], -1)
linear_response = stimulus_filtered_flat @ filter.flatten().T
nl_firing_rate = base_rate * np.exp(gain * linear_response + bias)

spike_probs = nl_firing_rate * dt
spike_train = rng_stimulus.random(spike_probs.shape) < spike_probs
time_ticks = np.linspace(0.001, duration_rec, spike_train.shape[0], endpoint=False)
spike_times = time_ticks[spike_train]

np.save('stimulus_ricker_1.npy', stimulus)
np.save('filter_ricker_1.npy', filter)
np.save('spike_train_ricker_1.npy', spike_train)

# Gabor

In [5]:
# image_size = (16, 16)

# base_rate = 5.0  # Base firing rate in Hz
# gain = 1.0      # Gain factor for nonlinearity
# bias = 0.0       # Bias term for nonlinearity
# dt = 0.003        # Time bin size in seconds (3 ms)
# random_seed_stim = 78465132

# duration_rec = 300 # Duration of recording in seconds
# # duration_rec = 3_000 # Duration of recording in seconds

# pattern_duration = 0.009 # Duration of each pattern in seconds
# # pattern duration should be prefearably a multiple of dt
# n_show_pattern = np.ceil(pattern_duration / dt).astype(int)
# n_patter_rec = np.ceil(duration_rec / pattern_duration).astype(int)

In [6]:
rng_stimulus = np.random.default_rng(random_seed_stim)
stimulus = rng_stimulus.integers(0, 2, (n_patter_rec, image_size[0], image_size[1]))
stimulus = 2 * stimulus - 1
# stimulus = rng_stimulus.uniform(0, 1, (n_patter_rec, image_size[0], image_size[1]))
stimulus = np.repeat(stimulus, n_show_pattern, axis=0)

filter = gabor(size=16, center=(6, 9), amplitude=2.0,
               sigma=2.0, theta=-np.pi/6, frequency=0.1, phase=0
        )

stimulus_filtered = stimulus * filter

stimulus_filtered_flat = stimulus_filtered.reshape(stimulus_filtered.shape[0], -1)
linear_response = stimulus_filtered_flat @ filter.flatten().T
nl_firing_rate = base_rate * np.exp(gain * linear_response + bias)

spike_probs = nl_firing_rate * dt
spike_train = rng_stimulus.random(spike_probs.shape) < spike_probs
time_ticks = np.linspace(0.001, duration_rec, spike_train.shape[0], endpoint=False)
spike_times = time_ticks[spike_train]

np.save('stimulus_gabor_1.npy', stimulus)
np.save('filter_gabor_1.npy', filter)
np.save('spike_train_gabor_1.npy', spike_train)